In [2]:
import logging, time, sys, os, json
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
import pandas as pd
from shapely import geometry
from shapely import wkt

logger = logging.getLogger(__name__)

In [3]:
all_data_dirs = {
            'cities-N':              os.path.join(os.getcwd(),'..','results_backup','simplify','cities_nodes_dataframe.csv'),
            'pipelines-cities':      os.path.join(os.getcwd(),'..','results_backup','output','cities_pipelines_edge_dataframe.csv'),
            'ports-cities':          os.path.join(os.getcwd(),'..','results_backup','output','cities_ports_edge_dataframe.csv'),
            'railways-cities':       os.path.join(os.getcwd(),'..','results_backup','output','cities_railways_edge_dataframe_alt.csv'),
            'coalmines-railways':    os.path.join(os.getcwd(),'..','results_backup','output','coal_mine_railway_edge_dataframe.csv'),
            'coalmines-N':           os.path.join(os.getcwd(),'..','results_backup','output','coal_mines_nodes_dataframe.csv'),
            'lng-N':                 os.path.join(os.getcwd(),'..','results_backup','output','lng_nodes_dataframe.csv',  ),
            'lng-pipelines':         os.path.join(os.getcwd(),'..','results_backup','output','lng_pipeline_edge_dataframe.csv'),
            'lng-shipping':          os.path.join(os.getcwd(),'..','results_backup','output','lng_shipping_route_edge_dataframe.csv'),
            'oilfields-pipelines':   os.path.join(os.getcwd(),'..','results_backup','output','oil_field_edge_dataframe.csv'),
            'oilfields-N':           os.path.join(os.getcwd(),'..','results_backup','oil_field_nodes_fixup.csv'),
            'pipelines-pipelines':   os.path.join(os.getcwd(),'..','results_backup','simplify','pipeline_edge_dataframe.csv'),
            'pipelines-N':           os.path.join(os.getcwd(),'..','results_backup','output','pipeline_node_dataframe.csv'),
            'ports-N':               os.path.join(os.getcwd(),'..','results_backup','output','port_node_dataframe.csv',  ),
            'ports-pipelines':       os.path.join(os.getcwd(),'..','results_backup','output','port_pipeline_edge_dataframe.csv'),
            'ports-railways':        os.path.join(os.getcwd(),'..','results_backup','output','port_railway_edge_dataframe.csv'),
            'ports-shipping':        os.path.join(os.getcwd(),'..','results_backup','output','port_ship_edge_dataframe.csv'),
            'powerstn-N':            os.path.join(os.getcwd(),'..','results_backup','output','power_station_nodes_dataframe.csv'),
            'powerstn-pipelines':    os.path.join(os.getcwd(),'..','results_backup','output','power_station_pipeline_edge_dataframe.csv'),
            'powerstn-railways':     os.path.join(os.getcwd(),'..','results_backup','output','power_station_railway_edge_dataframe.csv'),
            'procplant-N':           os.path.join(os.getcwd(),'..','results_backup','output','processing_plant_nodes_dataframe.csv'),
            'procplant-pipelines':   os.path.join(os.getcwd(),'..','results_backup','output','processing_plant_pipeline_edge_dataframe.csv'),
            'railways-railways':     os.path.join(os.getcwd(),'..','results_backup','output','railway_edge_dataframe.csv'),
            'railways-N':            os.path.join(os.getcwd(),'..','results_backup','output','railway_nodes_dataframe.csv'),
            'refineries-N':          os.path.join(os.getcwd(),'..','results_backup','output','refinery_nodes_dataframe.csv'),
            'refineries-pipelines':  os.path.join(os.getcwd(),'..','results_backup','output','refinery_pipeline_edge_dataframe.csv'),
            'shipping-shipping':     os.path.join(os.getcwd(),'..','results_backup','output','shipping_edge_dataframe.csv'),
            'shipping-N':            os.path.join(os.getcwd(),'..','results_backup','output','shipping_node_dataframe.csv'),
            'wellpads-N':            os.path.join(os.getcwd(),'..','results_backup','output','well_pad_nodes_dataframe.csv'),
            'wellpads-pipelines':    os.path.join(os.getcwd(),'..','results_backup','output','well_pad_pipeline_edge_dataframe.csv'),
        }

In [4]:
df_names = {
    'pipelines-N':{'ports-N':'ports-pipelines',
                'cities-N':'pipelines-cities',
                'powerstn-N':'powerstn-pipelines',
                'lng-N':'lng-pipelines',
                'procplant-N':'procplant-pipelines',
                'refineries-N':'refineries-pipelines',
                'wellpads-N':'wellpads-pipelines',
                'oilfields-N':'oilfields-pipelines'},
    'railways-N':{'cities-N':'railways-cities',
                'coalmines-N':'coalmines-railways',
                'powerstn-N':'powerstn-railways',
                'ports-N':'ports-railways'}
    }

### just do cities-railways

In [5]:
edge_df = pd.read_csv(all_data_dirs['railways-cities'])

In [6]:
r_df = pd.read_csv(all_data_dirs['railways-N'])

In [7]:
c_df = pd.read_csv(all_data_dirs['cities-N'])

In [9]:
r_df['coordinates'] = r_df['coordinates'].str.replace('(','[').str.replace(')',']')

In [12]:
c_df['coordinates'] = c_df['coordinates'].apply(json.loads)

In [13]:
r_df['coordinates'] = r_df['coordinates'].apply(json.loads)

In [15]:
c_df = c_df.set_index('CityNodeId:ID(CityNode)')

In [17]:
r_df = r_df.set_index('RailwayNodeID:ID(RailwayNode)')

In [19]:
edge_df.columns

Index(['Unnamed: 0', 'CityNode:END_ID(CityNode)',
       'RailwayNode:START_ID(RailwayNode)', ':TYPE', 'impedance'],
      dtype='object')

In [20]:
c_df['geometry'] = c_df['coordinates'].apply(geometry.LineString)
r_df['geometry'] = r_df['coordinates'].apply(geometry.LineString)

ValueError: Input [70.3302479906805, 37.76840637200007] is the wrong shape for a LineString

In [21]:
edge_df = edge_df.merge(r_df[['coordinates']], how='left', left_on='RailwayNode:START_ID(RailwayNode)', right_index=True)
edge_df.rename(columns={'coordinates':'start_coordinates'}, inplace=True)
edge_df = edge_df.merge(c_df[['coordinates']], how='left', left_on='CityNode:END_ID(CityNode)', right_index=True)
edge_df.rename(columns={'coordinates':'end_coordinates'}, inplace=True)

In [22]:
edge_df['start_geom'] = edge_df['start_coordinates'].apply(geometry.Point)
edge_df['end_geom'] = edge_df['end_coordinates'].apply(geometry.Point)

In [25]:
edge_df['dist'] = edge_df.apply(lambda row: haversine([row['start_geom'].x, row['start_geom'].y], [row['end_geom'].x, row['end_geom'].y]), axis=1)


In [26]:
edge_df

,Unnamed: 0,CityNode:END_ID(CityNode),RailwayNode:START_ID(RailwayNode),:TYPE,impedance,start_coordinates,end_coordinates,start_geom,end_geom,dist
0,76,city_node_76,railway_node_19.477934941.3163275,CITY_RAILWAY_CONNECTOR,0,"[19.4779349, 41.3163275]","[19.561707497710845, 41.29191763621817]",POINT (19.4779349 41.3163275),POINT (19.56170749771085 41.29191763621817),7506
1,76,city_node_76,railway_node_19.478668841.3161113,CITY_RAILWAY_CONNECTOR,0,"[19.4786688, 41.3161113]","[19.561707497710845, 41.29191763621817]",POINT (19.4786688 41.3161113),POINT (19.56170749771085 41.29191763621817),7440
2,76,city_node_76,railway_node_19.478761241.3160925,CITY_RAILWAY_CONNECTOR,0,"[19.4787612, 41.3160925]","[19.561707497710845, 41.29191763621817]",POINT (19.4787612 41.3160925),POINT (19.56170749771085 41.29191763621817),7432
3,76,city_node_76,railway_node_19.478817741.3161913,CITY_RAILWAY_CONNECTOR,0,"[19.4788177, 41.3161913]","[19.561707497710845, 41.29191763621817]",POINT (19.4788177 41.3161913),POINT (19.56170749771085 41.29191763621817),7431
4,76,city_node_76,railway_node_19.478977841.316047,CITY_RAILWAY_CONNECTOR,0,"[19.4789778, 41.316047]","[19.561707497710845, 41.29191763621817]",POINT (19.4789778 41.316047),POINT (19.56170749771085 41.29191763621817),7413
...,...,...,...,...,...,...,...,...,...,...
273573,13228,city_node_13228,railway_node_20.69148250016129342.245466499934146,CITY_RAILWAY_CONNECTOR,0,"[20.691482500161293, 42.245466499934146]","[20.752697840753427, 42.23426054348079]",POINT (20.69148250016129 42.24546649993415),POINT (20.75269784075343 42.23426054348079),5191
273574,13228,city_node_13228,railway_node_20.7033700003023442.23881750010726,CITY_RAILWAY_CONNECTOR,0,"[20.70337000030234, 42.23881750010726]","[20.752697840753427, 42.23426054348079]",POINT (20.70337000030234 42.23881750010726),POINT (20.75269784075343 42.23426054348079),4092
273575,13228,city_node_13228,railway_node_20.72016100003455542.22558649985989,CITY_RAILWAY_CONNECTOR,0,"[20.720161000034555, 42.22558649985989]","[20.752697840753427, 42.23426054348079]",POINT (20.72016100003455 42.22558649985989),POINT (20.75269784075343 42.23426054348079),2847
273576,13228,city_node_13228,railway_node_20.72608100070999342.22036549990785,CITY_RAILWAY_CONNECTOR,0,"[20.726081000709993, 42.22036549990785]","[20.752697840753427, 42.23426054348079]",POINT (20.72608100070999 42.22036549990785),POINT (20.75269784075343 42.23426054348079),2681


In [27]:
idxmins = edge_df.groupby('CityNode:END_ID(CityNode)').dist.idxmin().values

In [37]:
edge_df[['CityNode:END_ID(CityNode)','RailwayNode:START_ID(RailwayNode)', ':TYPE', 'impedance']].iloc[idxmins].to_csv('./../results_backup/simplify/cities_railways_edge_dataframe_alt.csv')

In [35]:
edge_df.iloc[idxmins]

,Unnamed: 0,CityNode:END_ID(CityNode),RailwayNode:START_ID(RailwayNode),:TYPE,impedance,start_coordinates,end_coordinates,start_geom,end_geom,dist
197791,10015,city_node_10015,railway_node_7.315613211.5523087,CITY_RAILWAY_CONNECTOR,0,"[7.3156132, 11.5523087]","[7.292772298769956, 11.547653311483895]",POINT (7.3156132 11.5523087),POINT (7.292772298769956 11.5476533114839),2542
197947,10026,city_node_10026,railway_node_7.915000411.3344347,CITY_RAILWAY_CONNECTOR,0,"[7.9150004, 11.3344347]","[7.892634819601731, 11.356874011632286]",POINT (7.9150004 11.3344347),POINT (7.892634819601731 11.35687401163229),3489
19367,1003,city_node_1003,railway_node_-47.25535829975959-22.78025063779...,CITY_RAILWAY_CONNECTOR,0,"[-47.25535829975959, -22.780250637797888]","[-47.25072293074102, -22.796014915825822]",POINT (-47.25535829975959 -22.78025063779789),POINT (-47.25072293074102 -22.79601491582582),1816
198053,10031,city_node_10031,railway_node_12.01354111.2736125,CITY_RAILWAY_CONNECTOR,0,"[12.013541, 11.2736125]","[12.027417548437134, 11.206786252505685]",POINT (12.013541 11.2736125),POINT (12.02741754843713 11.20678625250568),7583
198096,10039,city_node_10039,railway_node_7.616998811.2023995,CITY_RAILWAY_CONNECTOR,0,"[7.6169988, 11.2023995]","[7.630459807658388, 11.214537567738205]",POINT (7.6169988 11.2023995),POINT (7.630459807658388 11.2145375677382),1994
...,...,...,...,...,...,...,...,...,...,...
19365,998,city_node_998,railway_node_-42.06846788577255-22.47088889203686,CITY_RAILWAY_CONNECTOR,0,"[-42.06846788577255, -22.47088889203686]","[-42.06892883878045, -22.432401915329145]",POINT (-42.06846788577255 -22.47088889203686),POINT (-42.06892883878045 -22.43240191532914),4280
197319,9981,city_node_9981,railway_node_6.969706811.887333,CITY_RAILWAY_CONNECTOR,0,"[6.9697068, 11.887333]","[6.893405117769672, 11.846367254346195]",POINT (6.9697068 11.887333),POINT (6.893405117769672 11.84636725434619),9470
197336,9989,city_node_9989,railway_node_13.103209311.8344228,CITY_RAILWAY_CONNECTOR,0,"[13.1032093, 11.8344228]","[13.147584159650059, 11.766774687687745]",POINT (13.1032093 11.8344228),POINT (13.14758415965006 11.76677468768774),8939
197657,9990,city_node_9990,railway_node_8.592309812.0057055,CITY_RAILWAY_CONNECTOR,0,"[8.5923098, 12.0057055]","[8.593431162223446, 12.00241287418524]",POINT (8.592309800000001 12.0057055),POINT (8.593431162223446 12.00241287418524),386


In [ ]:
left_df['geometry'] = left_df['coordinates'].apply(geometry.Point)
right_df['geometry'] = right_df['coordinates'].apply(geometry.Point)

In [ ]:


logger.info('getting cols')
start_col = [cc for cc in edge_df.columns if 'START_ID' in cc][0]
end_col = [cc for cc in edge_df.columns if 'END_ID' in cc][0]

logger.info('merging')
edge_df = edge_df.merge(left_df[['geometry']], how='left', left_on=start_col, right_index=True)
edge_df.rename(columns={'geometry':'left_geom'}, inplace=True)
edge_df = edge_df.merge(right_df[['geometry']], how='left', left_on=end_col, right_index=True)
edge_df.rename(columns={'geometry':'right_geom'}, inplace=True)

logger.info('getting dists')
edge_df['dist'] = edge_df.apply(lambda row: haversine([row['left_geom'].x, row['left_geom'].y], [row['right_geom'].x, row['right_geom'].y]), axis=1)


In [4]:
def rep_pt(geom):
    return wkt.loads(geom).representative_point()

In [24]:
def haversine(coord1, coord2):
    import math
    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters)
    return meters

In [ ]:
#done = ['pipelines-ports']

In [6]:
def single_connection_simplify(left_df, right_df, edge_df):
    logger.info(f'casting to geometry...')   
    #print (left_df)
    left_df['geometry'] = left_df['coordinates'].apply(geometry.Point)
    right_df['geometry'] = right_df['coordinates'].apply(geometry.Point)
    
    logger.info('getting cols')
    start_col = [cc for cc in edge_df.columns if 'START_ID' in cc][0]
    end_col = [cc for cc in edge_df.columns if 'END_ID' in cc][0]
    
    logger.info('merging')
    edge_df = edge_df.merge(left_df[['geometry']], how='left', left_on=start_col, right_index=True)
    edge_df.rename(columns={'geometry':'left_geom'}, inplace=True)
    edge_df = edge_df.merge(right_df[['geometry']], how='left', left_on=end_col, right_index=True)
    edge_df.rename(columns={'geometry':'right_geom'}, inplace=True)
    
    logger.info('getting dists')
    edge_df['dist'] = edge_df.apply(lambda row: haversine([row['left_geom'].x, row['left_geom'].y], [row['right_geom'].x, row['right_geom'].y]), axis=1)
    return edge_df

In [7]:
done = []

In [8]:
for network, network_dict in df_names.items():
    net_asset = network.split('-')[0]
    logger.info(f'doing net asset: {net_asset}')
    network_df = pd.read_csv(all_data_dirs[network])
    
    network_df.coordinates = network_df.coordinates.str.replace('(','[').str.replace(')',']').apply(json.loads)
    
    network_id = [cc for cc in network_df.columns if 'ID' in cc][0]
    network_df.set_index(network_id, inplace=True)
    
    for kk,vv in network_dict.items():
        pt_asset  = kk.split('-')[0]
        if '-'.join([net_asset,pt_asset]) in done:
            logger.info(f'already done {net_asset}-{pt_asset}')
        else:
            logger.info(f'doing {net_asset}-{pt_asset}')


            pt_df = pd.read_csv(all_data_dirs[kk])

            pt_id = [cc for cc in pt_df.columns if 'ID' in cc][0]
            pt_df.set_index(pt_id, inplace=True)

            pt_df.coordinates = pt_df.coordinates.apply(json.loads)

            edge_df = pd.read_csv(all_data_dirs[vv])

            fname = all_data_dirs[vv].split('/')[-1]

            logger.info(f'all loaded and writing to fname {fname}')

            if net_asset==vv.split('-')[0]: #left
                logger.info(f'left: {net_asset}, {pt_asset}, edge_df')
                edge_df = single_connection_simplify(network_df, pt_df,edge_df)

            elif net_asset==vv.split('-')[1]: #right
                logger.info(f'right: {pt_asset}, {net_asset}, edge_df')
                edge_df = single_connection_simplify(pt_df, network_df, edge_df)
            else:
                print ('rugroh neither match')

            pt_col = [cc for cc in edge_df.columns if pt_df.index.name.split('(')[-1][:-1] in cc][0]
            logger.info(f'pt col: {pt_col}')

            logger.info(f'getting idx_mins')
            idxmins = edge_df.groupby(pt_col).dist.idxmin().values

            pre_len = len(edge_df)

            edge_df = edge_df.iloc[idxmins].drop(columns=['left_geom','right_geom','dist'])


            logger.info(f'reduced size from {pre_len} to {len(edge_df)}, writing to file')
            edge_df.to_csv('./../results_backup/simplify/'+fname)
            done.append('-'.join([net_asset,pt_asset]))
        
        
        
        

INFO:__main__:doing net asset: pipelines
INFO:__main__:doing pipelines-ports
INFO:__main__:all loaded and writing to fname port_pipeline_edge_dataframe.csv
INFO:__main__:right: ports, pipelines, edge_df
INFO:__main__:casting to geometry...
                                         facility_n     md_aquired  \
PortNodeId:ID(PortNode)                                              
port_node_0                               ArzewPort     Lucy Romeo   
port_node_1                              SkikdaPort     Lucy Romeo   
port_node_2                             CampanaPort     Lucy Romeo   
port_node_3                           LyttonOilPort     Lucy Romeo   
port_node_4                        SchwechatOilPort     Lucy Romeo   
...                                             ...            ...   
port_node_3697                       VALDEZHARBORAK  Devin Justman   
port_node_3698                            VANANDABC  Devin Justman   
port_node_3699                VINEYARDHAVENHARBORMA  Devin J

INFO:__main__:doing pipelines-cities
INFO:__main__:all loaded and writing to fname cities_pipelines_edge_dataframe.csv
INFO:__main__:left: pipelines, cities, edge_df
INFO:__main__:casting to geometry...
                                                                               coordinates  \
PipeNodeID:ID(PipelineNode)                                                                  
pipe_node_-151.5589190011921860.11453500057754    [-151.55891900119218, 60.11453500057754]   
pipe_node_-151.555557000382860.117760999896724    [-151.5555570003828, 60.117760999896724]   
pipe_node_-151.5501970000207260.120700000357594  [-151.55019700002072, 60.120700000357594]   
pipe_node_-151.5335289992113460.13117499949915    [-151.53352899921134, 60.13117499949915]   
pipe_node_-151.5212890013005260.137500000462786  [-151.52128900130052, 60.137500000462786]   
...                                                                                    ...   
pipe_node_178.00267010127737-38.6876417001960

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: PowerStationID:START_ID(PowerStation)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 2748232 to 10508, writing to file
INFO:__main__:doing pipelines-lng
INFO:__main__:all loaded and writing to fname lng_pipeline_edge_dataframe.csv
INFO:__main__:right: lng, pipelines, edge_df
INFO:__main__:casting to geometry...
                                 commodity                 facility_n  \
LngTerminalID:ID(LngTerminal)                                           
lng_terminal_0                 Natural Gas                   Canaport   
lng_terminal_1                 Natural Gas                    Unknown   
lng_terminal_2                 Natural Gas                    Unknown   
lng_terminal_3                 Natural Gas                    Unknown   
lng_terminal_4                 Natural Gas  ArzewGNL3ZLNGPlantAlgeria   
...                                    ...                   

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: LngTerminalID:START_ID(LngTerminal)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 49982 to 181, writing to file
INFO:__main__:doing pipelines-procplant
INFO:__main__:all loaded and writing to fname processing_plant_pipeline_edge_dataframe.csv
INFO:__main__:right: procplant, pipelines, edge_df
INFO:__main__:casting to geometry...
                                                       facility_n  \
ProcessingPlantID:ID(ProcessingPlant)                               
processing_plant_0                                       Elmworth   
processing_plant_1                               Encana Resthaven   
processing_plant_2                                       Elmworth   
processing_plant_3                           Gas Processing Plant   
processing_plant_4                                   Dawson Creek   
...                                                           ...   
p

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: ProcessingPlantID:START_ID(ProcessingPlant)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 509013 to 1688, writing to file
INFO:__main__:doing pipelines-refineries
INFO:__main__:all loaded and writing to fname refinery_pipeline_edge_dataframe.csv
INFO:__main__:right: refineries, pipelines, edge_df
INFO:__main__:casting to geometry...
                                                                 Commodity  \
RefineryID:ID(Refinery)                                                      
refinery_0               VirginNaphthaKeroseneGasoilGasolineHeavyfueloi...   
refinery_1                                     HeavyFuelOilAsphaltDiluents   
refinery_2                            GasolineKeroseneLubeOilNaphtaAsphalt   
refinery_3                            GasolineKeroseneLubeOilNaphtaAsphalt   
refinery_4                       LPGGasolineNaphtaKeroseneDieselOilFuelBTS   
...

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: RefineryID:START_ID(Refinery)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 189216 to 303, writing to file
INFO:__main__:doing pipelines-wellpads
INFO:__main__:all loaded and writing to fname well_pad_pipeline_edge_dataframe.csv
INFO:__main__:right: wellpads, pipelines, edge_df
INFO:__main__:casting to geometry...
                              facility_n     md_aquired  \
WellPadID:ID(WellPad)                                     
well_pad_0             UntitledPlacemark  Devin Justman   
well_pad_1             UntitledPlacemark  Devin Justman   
well_pad_2                        NKosso  Devin Justman   
well_pad_3                       Bilondo  Devin Justman   
well_pad_4                       Azurite  Devin Justman   
...                                  ...            ...   
well_pad_9840                         48  Devin Justman   
well_pad_9841                        

INFO:__main__:doing pipelines-oilfields
INFO:__main__:all loaded and writing to fname oil_field_edge_dataframe.csv
INFO:__main__:right: oilfields, pipelines, edge_df
INFO:__main__:casting to geometry...
                         Unnamed: 0  Unnamed: 0.1 commodity  \
OilFieldID:ID(OilField)                                       
oil_field_0                       0             0   Unknown   
oil_field_1                       1             1   Unknown   
oil_field_2                       2             2   Unknown   
oil_field_3                       3             3   Unknown   
oil_field_4                       4             4   Unknown   
...                             ...           ...       ...   
oil_field_25231               25231         25231   Unknown   
oil_field_25232               25232         25232   Unknown   
oil_field_25233               25233         25233   Unknown   
oil_field_25234               25234         25234   Unknown   
oil_field_25235               25235      

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: OilFieldID:START_ID(OilField)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 4485112 to 5998, writing to file
INFO:__main__:doing net asset: railways
INFO:__main__:doing railways-cities
INFO:__main__:all loaded and writing to fname cities_railways_edge_dataframe_alt.csv
INFO:__main__:left: railways, cities, edge_df
INFO:__main__:casting to geometry...
                                                                                  coordinates  \
RailwayNodeID:ID(RailwayNode)                                                                   
railway_node_-158.1521930002937821.387556000051557  [-158.15219300029378, 21.387556000051557]   
railway_node_-158.1521679997023921.38780900006117    [-158.15216799970239, 21.38780900006117]   
railway_node_-158.1521190004715421.3871660001145      [-158.15211900047154, 21.3871660001145]   
railway_node_-158.1518740000955721.38819900025

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: PowerStationID:START_ID(PowerStation)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 3390969 to 17876, writing to file
INFO:__main__:doing railways-ports
INFO:__main__:all loaded and writing to fname port_railway_edge_dataframe.csv
INFO:__main__:right: ports, railways, edge_df
INFO:__main__:casting to geometry...
                                         facility_n     md_aquired  \
PortNodeId:ID(PortNode)                                              
port_node_0                               ArzewPort     Lucy Romeo   
port_node_1                              SkikdaPort     Lucy Romeo   
port_node_2                             CampanaPort     Lucy Romeo   
port_node_3                           LyttonOilPort     Lucy Romeo   
port_node_4                        SchwechatOilPort     Lucy Romeo   
...                                             ...            ...   
port_node_

INFO:__main__:getting cols
INFO:__main__:merging
INFO:__main__:getting dists
INFO:__main__:pt col: PortNode:START_ID(PortNode)
INFO:__main__:getting idx_mins
INFO:__main__:reduced size from 657378 to 1928, writing to file


### Just do railways-cities

In [ ]:
edge_df = single_connection_simplify(ports_df, pipeline_df, edge_df)

In [ ]:
ports_df.index.name.split('(')[-1][:-1]

In [ ]:
pt_col = [cc for cc in edge_df.columns if ports_df.index.name.split('(')[-1][:-1] in cc][0]

In [ ]:
idxmins = edge_df.groupby(pt_col).dist.idxmin().values

In [ ]:
edge_df.iloc[idxmins].drop(columns=['left_geom','right_geom','dist'])

In [ ]:
ports_df.set_index(left_col)

In [ ]:
right_co
pipeline_df

In [ ]:
edges_df

In [ ]:
main_path = './../results_backup/output'
df_names = {
    'pipeline':['port_pipeline_edge_dataframe.csv',
                'cities_pipelines_edge_dataframe.csv',
                'power_station_pipeline_edge_dataframe.csv',
                'lng_pipeline_edge_dataframe.csv',
                'processing_plant_pipeline_edge_dataframe.csv',
                'refinery_pipeline_edge_dataframe.csv',
                'well_pad_pipeline_edge_dataframe.csv',
                'oil_field_edge_dataframe.csv'],
    'railway':['cities_railways_edge_dataframe.csv',
                'coal_mine_railway_edge_dataframe.csv',
                'power_station_railway_edge_dataframe.csv',
                'port_railway_edge_dataframe.csv']
    }